In [ ]:
import pandas as pd
import os
import csv
from functools import reduce

In [ ]:
# Read the contents of the text file
with open('/content/drive/MyDrive/H10-PublicHealth/ARCOS/dataCensus/quarter_to_months.txt', 'r') as file:
    dictionary_str = file.read()

# Parse the string into a dictionary
quarter_to_months = eval(dictionary_str)

In [ ]:
# DataFrame with County FIPS code for all counties in the 50 US States + DC
cnty = pd.read_csv("/content/drive/MyDrive/H10-PublicHealth/ARCOS/dataCensus/countyfips.csv")
  # generated in 2024-03-09-Homophily-Dissimilarity.ipynb - From Serena's other projects

## Andrew: Update the "yearquarter" variable and execute the code below

In [ ]:
yearquarter = "2019Q1" # from 2006Q1 to 2019Q4

In [ ]:
# Initiate a lank list to store year-month pairs
rows = []

# Iterate over each row in the DataFrame
for index, row in cnty.iterrows():
    # Multiply each row by three
    for i in range(3):
        year_months = quarter_to_months[yearquarter] # obtain the values corresponding to the key (which is the specified "yearquarter")

        row = {
            'FIPS': row['FIPS'],
            'buyer_cnty_state': row['buyer_cnty_state'],
            'year_month': year_months[i]
        }
        rows.append(row)

# Create a new df from the list of new rows
cnty_df = pd.DataFrame(rows)
cnty_df

In [ ]:
# Define the column headers to be used
column_headers = ['transaction_code', 'transaction_date', 'drug_code', 'buyer_bus_act', 'buyer_county', 'buyer_state', 'calc_base_wt_in_gm']

# Define the directory containing the files
directory = f"/content/drive/MyDrive/H10-PublicHealth/ARCOS/dataARCOS/dataRaw/" + yearquarter + "/"
files = os.listdir(directory)

# Initialize an empty DataFrame to store the final result
result_df = pd.DataFrame()

for file in files:
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, file), usecols = column_headers)

        df["buyer_cnty_state"] = df["buyer_county"]+ ", " + df["buyer_state"]

        mask = (
            (df["transaction_code"] == "S") &
            (df['drug_code'].isin(["9064", "9250B"])) &
            (~df['buyer_bus_act'].isin(["MLP-ANIMAL SHELTER", "CANINE HANDLER", "TEACHING INSTITUTION"]))
        )

        # Apply the mask to the DataFrame
        filtered_df = df[mask]

        # Creating a new column, "year_month" : the first 7 characters (year and month) from the "transaction_date" column.
        filtered_df.loc[:, "year_month"] = filtered_df["transaction_date"].str[:7]

        # Filtered DataFrames for drug_code "9064" (BUPRENORPHINE) and "9250B" (METHADONE)
        df_9064_filtered = filtered_df[filtered_df["drug_code"] == "9064"]
        df_9250B_filtered = filtered_df[filtered_df["drug_code"] == "9250B"]

        # Grouping and aggregation for drug_code "9064"
        df_9064_grouped = df_9064_filtered.groupby(['buyer_cnty_state', 'year_month']).agg(
            BUPRENORPHINE_transaction =('drug_code', 'size'),
            BUPRENORPHINE_wt_in_gm=('calc_base_wt_in_gm', 'sum')
        ).reset_index()

        # Grouping and aggregation for drug_code "9250B"
        df_9250B_grouped = df_9250B_filtered.groupby(['buyer_cnty_state', 'year_month']).agg(
            METHADONE_transaction =('drug_code', 'size'),
            METHADONE_wt_in_gm=('calc_base_wt_in_gm', 'sum')
        ).reset_index()

        # Merge the two DataFrames on 'buyer_cnty_state' and 'year_month'
        df_sum = pd.merge(df_9064_grouped, df_9250B_grouped, on=['buyer_cnty_state', 'year_month'], how='inner')

        # Merge the current result with previous results
        result_df = pd.merge(cnty_df, df_sum, on=['buyer_cnty_state', 'year_month'], how='left')

        # Fill NaN values (== no transactions) with 0
        result_df.fillna(0, inplace=True)

        # Save the DataFrame as a CSV File
        result_df.to_csv(f"/content/drive/MyDrive/H10-PublicHealth/ARCOS/dataARCOS/CountyQuarterly/arcos-cnty-{yearquarter}.csv", index=False)